In [1]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import ConfigurableField
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_ollama.llms import OllamaLLM
from typing import Tuple, List, Optional
from langchain.chains import GraphCypherQAChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain import hub
from langchain.schema.runnable import RunnablePassthrough
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.prompts import ChatMessagePromptTemplate, PromptTemplate

from neo4j import GraphDatabase
from langchain_community.vectorstores import Neo4jVector
from langchain_community.graphs import Neo4jGraph
from langchain.vectorstores import FAISS

import pickle
import os

In [2]:
#loading the embedding model from huggingface
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(
  model_name=embedding_model_name,
  model_kwargs=model_kwargs
)

/tmp/ipykernel_34152/269612671.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/home/mouli/miniconda3/envs/gastro/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
# Load from local storage
persisted_vectorstore = FAISS.load_local("faiss_index_", embeddings,allow_dangerous_deserialization=True)

In [4]:
#creating a retriever on top of database
retriever = persisted_vectorstore.as_retriever()

In [5]:
fusion_template = """
        Task: You are an assistant that generates multiple variations of a given question. 
        For each variation, maintain the original intent of the question, but change the phrasing, structure, 
        or tone to create a diverse set of queries.

Generate 5-7 variations that cover:

Synonym replacements while keeping the question concise.
Alternative structures, such as rephrasing into "why," "how," or "what" forms if relevant.
Casual and formal tones.
Slightly more specific or broader wording.
Examples:

Original Question: "What is the impact of inflation on the stock market?"
Variations:
"How does inflation affect stock prices?"
"What are the effects of inflation on the stock market?"
"In what ways does inflation influence stock market trends?"
"Could inflation lead to changes in stock market values?"
"How does rising inflation impact the performance of stocks?"
"What influence does inflation have on market prices?"
"What happens to stock prices when inflation increases?"

the final answer should be a python list: [
   "How does inflation affect stock prices?",
    "What are the effects of inflation on the stock market?",
    "In what ways does inflation influence stock market trends?",
    "Could inflation lead to changes in stock market values?",
    "How does rising inflation impact the performance of stocks?",
    "What influence does inflation have on market prices?",
    "What happens to stock prices when inflation increases?"
]

Now generate the list of variations for the given question
        """

In [6]:
original_query = "What are the new AI regulations in Europe"

In [7]:
prompt = ChatPromptTemplate(input_variables=['original_query'],
                            messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[],template=fusion_template)),
                            HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['original_query'], template='Generate multiple search queries related to: {question} \n OUTPUT (7 queries):'))])

In [8]:
model = OllamaLLM(model="llama3.2")

In [9]:
generate_queries = (
    prompt | model | StrOutputParser() | (lambda x: x.split("\n"))
)

In [10]:
print(generate_queries)

first=ChatPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='\n        Task: You are an assistant that generates multiple variations of a given question. \n        For each variation, maintain the original intent of the question, but change the phrasing, structure, \n        or tone to create a diverse set of queries.\n\nGenerate 5-7 variations that cover:\n\nSynonym replacements while keeping the question concise.\nAlternative structures, such as rephrasing into "why," "how," or "what" forms if relevant.\nCasual and formal tones.\nSlightly more specific or broader wording.\nExamples:\n\nOriginal Question: "What is the impact of inflation on the stock market?"\nVariations:\n"How does inflation affect stock prices?"\n"What are the effects of inflation on the stock market?"\n"In what ways does inflation influence stock market tren

In [11]:
from langchain.load import dumps, loads


def reciprocal_rank_fusion(results: list[list], k=60):
    fused_scores = {}
    for docs in results:
        # Assumes the docs are returned in sorted order of relevance
        for rank, doc in enumerate(docs):
            doc_str = dumps(doc)
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            previous_score = fused_scores[doc_str]
            fused_scores[doc_str] += 1 / (rank + k)

    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]
    return reranked_results

In [12]:
ragfusion_chain = generate_queries | retriever.map() | reciprocal_rank_fusion

In [13]:
import langchain
langchain.debug = True

In [14]:
ragfusion_chain.input_schema.schema()

{'properties': {'question': {'title': 'Question', 'type': 'string'}},
 'required': ['question'],
 'title': 'PromptInput',
 'type': 'object'}

In [ ]:
ragfusion_chain.invoke({"question": original_query})

In [16]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

full_rag_fusion_chain = (
    {
        "context": ragfusion_chain,
        "question": RunnablePassthrough()
    }
    | prompt
    | model
    | StrOutputParser()
)

In [17]:
full_rag_fusion_chain.input_schema.schema()

{'properties': {'question': {'title': 'Question', 'type': 'string'},
  'root': {'title': 'Root'}},
 'required': ['question', 'root'],
 'title': 'RunnableParallel<context,question>Input',
 'type': 'object'}

In [ ]:
result = full_rag_fusion_chain.invoke({"question": "How long does it typically take for a gastric ulcer to heal?"})

In [19]:
print(result)

Based on the provided clinical guidelines, the healing time for gastric ulcers is not explicitly mentioned. However, the general treatment approach for gastric ulcers and other gastrointestinal bleeding conditions is discussed in the guidelines.

For patients with gastric varices, endoscopic injection of N-butyl-2-cyanoacrylate is offered to control upper gastrointestinal bleeding (Section 1.5.5). The duration of this treatment is not specified, but it is likely that the patient will need ongoing monitoring and potential repeat treatments as needed.

In general, the healing time for gastric ulcers can vary depending on several factors, such as the severity of the ulcer, the effectiveness of treatment, and individual patient response. However, with appropriate medical management, including proton pump inhibitors (PPIs) or other anti-secretory medications, gastric ulcers typically heal within a few weeks to months.

In the absence of specific information in the provided guidelines, it is